In [1]:
from IPython.display import Audio, display
import numpy as np
import pandas as pd
from jiwer import wer, cer
from tqdm import tqdm

In [2]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="7"
import glob
import torch
import matplotlib.pyplot as pltm
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
longformdata = pd.read_csv("/app/dataset/metadata/annoated.csv",delimiter="	")

In [4]:
from configs.wav2vec2_characterwise_pretrained_ctc import Configs
CFG = Configs(longformdata.file.apply(lambda x: os.path.join("/app/dataset/examples",x)),longformdata.sentence)

inference mode is on


In [5]:
CFG.load_state_dict("/app/bengali-speech-recognition/workdir/wav2vec2_characterlevel_pretrained_ctcloss_noaugs/bestmodel_wer.pkl")
CFG.model.cuda()
1

loading model checkpoint from epoch:  20000


1

In [6]:
def sliding_window(tensor, window_size, overlap):
    """
    Slices the input tensor with a sliding window of given size and overlap.
    
    Args:
        tensor (torch.Tensor): The input tensor of shape (points,).
        window_size (int): The size of the sliding window.
        overlap (int): The overlap between consecutive windows.
        
    Returns:
        torch.Tensor: A tensor containing the sliced windows stacked along the 0 dimension.
    """
    step = window_size - overlap
    slices = []
    
    for start in range(0, len(tensor) - window_size + 1, step):
        end = start + window_size
        slices.append(tensor[start:end])
    
    # Handle the final feature and pad it with zeros to match window_size
    if len(tensor) % step != 0:
        last_slice = tensor[-window_size:]
        slices.append(torch.nn.functional.pad(last_slice, (0, window_size - len(last_slice))))
    
    return torch.stack(slices, dim=0)


In [7]:
all_truth= []
all_preds = []
OVERLAP=1
for feature,truth in tqdm(CFG.inference_dataset):
    with torch.no_grad():
        feature = sliding_window(feature,CFG.MAX_AUDIO_LENGTH,OVERLAP).cuda()
        preds = CFG.model(feature)
        all_indices = torch.argmax(preds.detach().cpu(), dim=-1)
    generated = []
    for indices in all_indices:
        indices = torch.unique_consecutive(indices, dim=-1)
        indices = indices[indices != CFG.BLANK_TOKEN]
        generated.extend(CFG.tokenizer.decode_torch_inference(indices))
    prediction = "".join(generated)
    all_truth.append(CFG.tokenizer.decode_torch_inference(truth))
    all_preds.append(prediction)

100%|██████████| 17/17 [00:06<00:00,  2.69it/s]


In [8]:
longformdata["predictions"] = all_preds
longformdata["truth"] = all_truth
longformdata["wer"] = [wer(a,b) for a,b in zip(all_truth,all_preds)]
longformdata["cer"] = [cer(a,b) for a,b in zip(all_truth,all_preds)]


In [12]:
longformdata.wer.mean(),longformdata.cer.mean()

(0.8809398167745288, 0.549489238711458)

In [9]:
longformdata.wer.mean(),longformdata.cer.mean()

(0.8483031106888939, 0.4413356898550691)